In [1]:
import pandas as pd
import numpy as np
import os
import re
import datetime as dt

In [2]:
df_names = pd.read_excel('Manual.xlsx', sheet_name="Name", index_col=None)

In [3]:
df_INN = pd.read_excel('Manual.xlsx', sheet_name="TaxID", index_col=None, dtype=({'ИНН':str}))

In [4]:
df_ext = pd.read_excel('Manual.xlsx', sheet_name="Ext", index_col=None, dtype=({'Индекс':str}))

In [5]:
if df_names.empty == False:   
    #Убираем символы переноса строки
    df_names['Имя'] = df_names['Имя'].str.strip('\n\r\"\(\)')
    #Заменяем недопустимые символы процентом
    df_names['Имя'] = df_names['Имя'].str.replace('[^a-zA-Z0-9\\s\\-.\$/:+%№]+', '%', regex = True)
    #Обрезаем (имена с учетом маски до 28 символов, города до 13 символов)
    df_names.loc[df_names['Имя'].str.len()>28, 'Имя']= df_names['Имя'].str[:28]
    #Ставим маски для имен больше 7 символов
    df_names.loc[df_names['Имя'].str.len()>=8, 'Имя']= '%' + df_names['Имя'] + '%'
    #Заменяем несколько процентов подряд одним процентом
    df_names['Имя'] = df_names['Имя'].str.replace('%{2,}', '%', regex = True)

In [6]:
if df_ext.empty == False:     
    #Убираем символы переноса строки
    df_ext['Имя'] = df_ext['Имя'].str.strip('\n\r\"\(\)')
    df_ext['Город'] = df_ext['Город'].str.strip('\n\r \"\(\)')
    #Заменяем недопустимые символы процентом
    df_ext['Имя'] = df_ext['Имя'].str.replace('[^a-zA-Z0-9\\s\\-.\$/:+%№]+', '%', regex = True)
    df_ext['Город'] = df_ext['Город'].str.replace('[^a-zA-Z0-9\\s\\-.\$/:+%№]+', '%', regex = True)
    #Обрезаем (имена с учетом маски до 28 символов, города до 13 символов)
    df_ext.loc[df_ext['Имя'].str.len()>28, 'Имя']= df_ext['Имя'].str[:28]
    df_ext.loc[df_ext['Город'].str.len()>13, 'Город']= df_ext['Город'].str[:13]
    #Ставим маски для имен больше 7 символов
    df_ext.loc[df_ext['Имя'].str.len()>=8, 'Имя'] = '%' + df_ext['Имя'] + '%'
    #Заменяем несколько процентов подряд одним процентом
    df_ext['Имя'] = df_ext['Имя'].str.replace('%{2,}', '%', regex = True)

In [7]:
#Заголовки
df_head = pd.DataFrame(columns=['FIXED VALUE', 'TABLE'])
df_head['TABLE'] = ['CSAC$MERCHANTS_GROUP', 'CSAC$MERCHANTS_EXT_GROUP']
df_head['FIXED VALUE'] = 'H'

In [8]:
if df_names.empty == False: 
    df_names_UFL = pd.DataFrame(columns=['Action', 'GROUP_ID', 'ATTRIBUTE', 'VALUE'])
    df_names_UFL['Action'] = df_names['Добавить/удалить (A/D)']
    df_names_UFL['GROUP_ID'] = df_names['Группа']
    df_names_UFL['VALUE'] = df_names['Имя']
    df_names_UFL['ATTRIBUTE'] = 'NAME'
    df_UFL = df_names_UFL

In [9]:
if df_INN.empty == False: 
    df_INN_UFL = pd.DataFrame(columns=['Action', 'GROUP_ID', 'ATTRIBUTE', 'VALUE'])
    df_INN_UFL['Action'] = df_INN['Добавить/удалить (A/D)']
    df_INN_UFL['VALUE'] = df_INN['ИНН']
    df_INN_UFL[['GROUP_ID', 'ATTRIBUTE']] = [202, 'TAXID']
    df_UFL = df_INN_UFL

In [10]:
#Список для короткого файла
if (df_names.empty == False) & (df_INN.empty == False):
    df_UFL = pd.concat([df_INN_UFL, df_names_UFL])

In [11]:
#Для расширенного файла
df_ext.columns=['Action', 'GROUP_ID', 'NAME', 'LOCATION', 'MCC', 'MID', 'ZIP', 'TAXID', 'COUNTRY', 'CITY']

In [12]:
#Сохраняем все в csv
n = int(input("Введите порядковый номер загрузки файлов в течение дня\n"))
if (df_names.empty == False | df_INN.empty == False) & df_ext.empty == False:
    filename1 = 'MGP_' + pd.to_datetime("today").strftime("%Y%m%d") + '_' + str(2*n-1).zfill(3) + '.csv'
    filename2 = 'MGP_' + pd.to_datetime("today").strftime("%Y%m%d") + '_' + str(2*n).zfill(3) + '.csv'
    df_head.loc[[0]].to_csv(filename1, sep=';', encoding='UTF-8', header=False, index=False, mode = 'w')
    df_UFL.to_csv(filename1, sep=';', encoding='UTF-8', header=False, index=False, mode = 'a')
    df_head.loc[[1]].to_csv(filename2, sep=';', encoding='UTF-8', header=False, index=False, mode = 'w')
    df_ext.to_csv(filename2, sep=';', encoding='UTF-8', header=False, index=False, mode = 'a')
else:
    filename = 'MGP_' + pd.to_datetime("today").strftime("%Y%m%d") + '_' + str(2*n-1).zfill(3) + '.csv'
    if df_names.empty == False | df_INN.empty == False:
        df_head.loc[[0]].to_csv(filename, sep=';', encoding='UTF-8', header=False, index=False, mode = 'w')
        df_UFL.to_csv(filename, sep=';', encoding='UTF-8', header=False, index=False, mode = 'a')
    elif df_ext.empty == False:
        df_head.loc[[1]].to_csv(filename, sep=';', encoding='UTF-8', header=False, index=False, mode = 'w')
        df_ext.to_csv(filename, sep=';', encoding='UTF-8', header=False, index=False, mode = 'a')

Введите порядковый номер загрузки файлов в течение дня
1
